## Pakiet Pandas

Pakiet Pandas to prawdopodobnie najważniejsza bibliotego języka Python dla analityka danych. 

Biblioteka pozwala na tworzenie zbiorów danych (ang. *DataFrame*) analogicznych do tych wpisanych w arkuszach Excel. Pakiet umożliwia w przystępny sposób wykorzystywać operacje na całych kolumnach danych bez pisania rozległych pętli. 

### Tworzenie DataFrame
Jak stworzyć obiekt *DataFrame*? W przypadku programów pobierających dane np. ze stron www czy API najczęsciej będizemy konwertowac słownik.

Wygląda to tak:

In [ ]:
import pandas as pd

# Dane składujemy w kilku listach - jest to mocno niekomfortowe. Ich edytowanie czy nawet przeglądanie jest trudne.
nazwiska = ["Neymar", "Kylian Mbappé", "Philippe Coutinho"]	
pozycja = ["Lewy napastnik", "Środkowy napastnik", "Ofensywny pomocnik"]
odstepne = [222, 145, 135]

# Listy możemy zorganizować w słownik - nie da nam on jednak dużej funkcjonalności
slownik_dane = {
    "Nazwisko": nazwiska,
    "Pozycja": pozycja,
    "Transfer": odstepne    
}

# Tutaj pojawia się pakiet pandas:
pandas_dane = pd.DataFrame(slownik_dane)

print(pandas_dane)


*DataFrame* możemy stworzyć też z każdej listy obiektów np. listy list bądź listy krotek. Takie rozwiązanie wymagać będzie jednak zdefiniowania nazw kolumn:

In [ ]:
dane_listaF1 = [("Max Verstappen",	"Holandia",	"Red Bull Racing",	351.5), 
                ("Lewis Hamilton",	"W. Brytania",	"Mercedes",	343.5), 
                ("Valtteri Bottas",	"Finlandia",	"Mercedes",	203)]

dane_pandasF1 = pd.DataFrame.from_records(dane_listaF1, columns=['Kierowca', 'Kraj', 'Team', 'Pkt'])
print(dane_pandasF1)


### Wczytywanie danych z plików
Prawdziwym benefitem jest jednak możliwość prostego wczytania danych z pliku. Biblioteka pandas umożliwia otworzenie danych z: 
* plików CSV 
* plików Excel, Open Office
* większości pakietów statystycznych np. SPSS, Stata.

Artykuł przeglądowy można znaleźć [tutaj](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) 

Posłużymy się przykładem - ściągniemy plik CSV z Githuba.

Instrukcja head wyświetli nam kilka pierwszych wierszy z tabeli.

In [ ]:
import pandas as pd
URL = "https://raw.githubusercontent.com/jakubrybacki/Karowa_Python_Introduction/main/Zajecia%208/SerieA_2020_2021.csv"

dane_SerieA = pd.read_csv(URL)
dane_SerieA.head(10)

Zawartość tabeli można wyświetlić komendą info:

In [ ]:
dane_SerieA.info()

Ewentualnie w jeszcze krótszej formie:

In [ ]:
dane_SerieA.keys()

### Dane fromat Wide / Long

Dane pobierane z internetu mogą mieć długą formułe - tak jest z naszym przykładowym plikiem z Serie A. 

Biblioteka pandas pozwala to jednak szybko przekształcić na plik ułożony w wierszach. Przykład poniżej.

In [ ]:
daneSerieA_wide = dane_SerieA.pivot(index='MatchID', columns='StatName', values='StatValue')
daneSerieA_wide.info()


In [ ]:
daneSerieA_wide.keys()

Obejrzyjmy też pierwszych 10 wierszy po zmianie:

In [ ]:
daneSerieA_wide.head(10)


### Wybieranie poszczególnych kolumn

Obecnie posiadamy bardzo duży pakiet informacji, a nie potrzebujemy całej jego zawartości. Załóżmy, że zależy nam tylko na kilku informacjach o bramkach i strzałach. 

Przeniesiemy je do nowego obiektu:

In [ ]:
snapSerieA = daneSerieA_wide[['HomeTeam', 'AwayTeam', 'Goals', "Home: Strzały na bramkę", "Away: Strzały na bramkę"]]
snapSerieA.info()

### Szybkie konwersje typów
Dane pobierane z CSV czy z sieci web będą utrzymywane w formie napisu (str) Pakiet pandas umożliwia szybką konwersję.

Chcemy przekształcić informację o strzałach na bramkę do typu numerycznego:

In [ ]:
snapSerieA[["Home: Strzały na bramkę", "Away: Strzały na bramkę"]] = snapSerieA[["Home: Strzały na bramkę", "Away: Strzały na bramkę"]].astype(float)
snapSerieA.info()

### Modyfikowanie zawartości kolumn - proste obliczenia
Pandas oferuje możliwość bardzo szybkiego wykonywania obliczeń na podstawie posiadanych serii danych.

Przypuśćmy, że chcemy stworzyć serie róznica strzałów na podstawie danych z dwóch serii (strzały gospodarza, strzały gościa). 

Na początek obejrzyjmy wyniki z pierwszych 5 spotkań:

In [ ]:
snapSerieA[["Home: Strzały na bramkę", "Away: Strzały na bramkę"]].head(5)

Stworzenie nowej serii danych zawierających różnicę odbywa się podobnie jak w słowniku - wystarczy tylko jedna komenda:

In [ ]:
snapSerieA["Różnica: Strzały na bramkę"] = snapSerieA["Home: Strzały na bramkę"] - snapSerieA["Away: Strzały na bramkę"]
snapSerieA[["Home: Strzały na bramkę", "Away: Strzały na bramkę", "Różnica: Strzały na bramkę"]].head(5)

### Modyfikowanie zawartości kolumn - operacje na tekstach
Pakiet pandas posiada zaimplementowane wszystkie główne metody typu string do przeprowadzania na całej kolumnie danych. Aby je wykonać należy dopisać do nazwy kolumny *str.nazwa_metody*

Przetrenujemy wykorzystanie metod *replace*, *split* i *strip* na kolumnie Goals. Najpierw wydrukujmy zawartość - zobaczymy m.in znaki końca linii, separator w formie myślnika. 

In [ ]:
snapSerieA["Goals"].head(5)

Najpierw usuwamy znaki końca lini (\n) - wykorzystamy metodę *replace*:

In [ ]:
snapSerieA["Goals"] = snapSerieA["Goals"].str.replace("\n", "")
snapSerieA["Goals"].head(5)

Metodą *split* usuniemy separator w postaci myślnika. 

W *DataFrame* pojawia się  w niej dodatkowy argument expand:
* Domyślenie jest on ustawiony na *False* - wtedy funkcja zwraca listę zupełnie jak bazowa metoda string
* Ustawienie go na *True* spowoduje zwrócenie rozpakowanych elementów listy - wykorzystam to, aby od razu powołać dwie nowe kolumny.

In [ ]:
snapSerieA[["Home: Goals", "Away: Goals"]] = snapSerieA["Goals"].str.split("-", expand = True)
snapSerieA[["Goals", "Home: Goals", "Away: Goals"]].head(5)

Na koniec wyczyścimy białe znaki i przemianujemy wartości bramek na numeryczne:

In [ ]:
snapSerieA["Home: Goals"] = snapSerieA["Home: Goals"].str.strip().astype(float)
snapSerieA["Away: Goals"] = snapSerieA["Away: Goals"].str.strip().astype(float)

snapSerieA.info()

### Analiza - podstawowe statystyki opisowe
Pakiet Pandas posiada gotowy szybki wydruk statystyk opisowych przy pomocy metody *describe*:

In [ ]:
snapSerieA[["Home: Goals", "Away: Goals", "Różnica: Strzały na bramkę"]].describe()

Możemy przeprowadzić też dużą gamę obliczeń wykorzystując gotowe metody. Pełny spis znajdziemy w [dokumentacji](https://pandas.pydata.org/docs/reference/frame.html) 

Kilka przykładów poniżej.

Moda:

In [ ]:
snapSerieA[["Home: Goals", "Away: Goals", "Różnica: Strzały na bramkę"]].mode(dropna=True)

Odchylenie standardowe:

In [ ]:
snapSerieA[["Home: Goals", "Away: Goals", "Różnica: Strzały na bramkę"]].std()

Macierz Korelacji: 

In [ ]:
snapSerieA[["Home: Goals", "Away: Goals", "Różnica: Strzały na bramkę"]].corr()

### Wizualizacja danych na wykresie
Wykonaliśmy kilka obliczeń - warto je przedstawić na wykresie. Najbardziej praktycznym rozwiązaniem będzie w tym przypadku prosty histogram. 

Pandas korzysta z biblioteki *MatPlotLib*, która pozwala na bardzo szczegółowe określanie treści wykresów. 

My stworzymy je z domyślnymi wartościami - będzie to trochę mało okazałe.



In [ ]:
snapSerieA["Home: Goals"].hist(bins=7)

In [ ]:
snapSerieA["Różnica: Strzały na bramkę"].hist()

### Eksport wyników do pliku

Pakiet Pandas pozwala szybko zapisać treść do pliku za pomocą gotowych metod. Zapiszemy nasz *DataFrame* do miejsca z którego wykonujemy skrypt.

Najpierw wydobądźmy nazwę ścieżki

In [ ]:
import os
filePath = os.getcwd() + "/" 
print(filePath)

Zapiszmy plik - warto tu zwrócić uwagę na kodowanie. Standardowo pliki zapisywane są w uniwersalnym formacie UTF-8. Takie rozwiązanie nie jest kompatybilne z pakietem MS Office.

Aby temu przeciwdziałać można dodać argument *encoding*:

In [ ]:
snapSerieA.to_csv(filePath + "SerieASnap_UTF.csv", index = False)
snapSerieA.to_csv(filePath + "SerieASnap_Windows.csv", index = False, encoding="windows-1250")